## Wav2Vec

Wav2Vec 2.0, which has the potential to solve audio-related Natural Language Processing (NLP) tasks.

In [ ]:
!pip install torch

### Convert Audio file to .wav file format

In [1]:
import sys
sys.path.append('/usr/local/Cellar/ffmpeg')

In [2]:
from pydub import AudioSegment

In [3]:
mp3_audio = AudioSegment.from_file("speech.mp3", format="mp3")
mp3_audio.export("speech_Converted.wav", format="wav")

<_io.BufferedRandom name='speech_Converted.wav'>

In [4]:
mp3_audio = AudioSegment.from_file(r"speech_Converted.wav", format="wav")
print(len(mp3_audio)/(1000*60))
# 12 Minutes audio breaks into 3 minutes 4 audio files (slicingis done by milliseconds)

counter_audio = 180
split_audio = [mp3_audio[:180*1000]]
for i in range(4):
    split_audio.append(mp3_audio[counter_audio*1000:(counter_audio+180)*1000])
    counter_audio += 180

count = 0
# # lets save it!
for count, audio_object in enumerate(split_audio):
    count += 1
    with open(f"{count}_audi_file.wav", 'wb') as out_f:
        audio_object.export(out_f, format='wav')

2.261433333333333


## Predict Text from Audio

Wav2vec 2.0 model to get the text from audio. we can achieve this by looping the audio files that earlier i have converted into smaller audio files.

In each loop, predict the text from audio and store it into the collection_of_text variable. At last, add all the text in the collection_of_text variable and store it in the final_complete_speech variable.

In [5]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

# load model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# The base model pretrained and fine-tuned on 960 hours of Librispeech on 16kHz sampled speech audio.
# When using the model make sure that your speech input is also sampled at 16Khz.

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/Users/ankit/opt/anaconda3/envs/wav_vec/lib/python3.9/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:746: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import numpy as np
# load any audio file of your choice
collection_of_text = []
for i in range(1):

    speech, rate = librosa.load(f"{i+1}_audi_file.wav", sr=16000)

    input_values = tokenizer(speech, return_tensors='pt').input_values
    # Store logits (non-normalized predictions)
    with torch.no_grad():
        logits = model(input_values).logits

    # Store predicted id's
    predicted_ids = torch.argmax(logits, dim=-1)
    # decode the audio to generate text
    # Passing the prediction to the tokenzer decode to get the transcription
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    # transcriptions = tokenizer.decode(predicted_ids[0])
    print(transcription)
    collection_of_text.append(transcription)

THE DAY OF THE SPEECH I COULDN'T EAT ALL DAY I WAS SO NERVOUS FOR HIM I WEN IT IT TO GO JUST PERFECTLY HE STARTS OFF AND I GET O TELL YOU IAM SWEAT AND I FEEL LIKE I'M IN A SANA I STAND HERE KNOWING THAT MY STORY IS PART OF THE LARGER AMERICAN STORY THAT I OWE ADEBT TO ALL OF THOSE WHO CAME BEFORE ME AND THAT NO OTHER COUNTRY ON EARTH IS MY STORY EVEN POSIBLIPEEPLE GET THIN TO ITAND SLOWLY BUT SURELY GOES FROM GOOD TO GREAT IT IS TAT UNDOMENTAL BELIEF I AM MY ROTHER'S KEEPER I AM MY THI TERS KEEPER THAT MAKES THIS OT REWORK AND THEN I START TO GET THIS FEELING OF RELIEF THAT HE'S DOING A GOOD JOB AND THEN WHEN HE JUST HIT A HOME RUN WITH IT TO NIGHT THERE IS NOT A LIBERAL AMERICA AND A CONSERVATIVE AMERICA THERE IS THE UNITED STATES OF AMERICA A A E MERI A AN A MY O MERIO A OINAL MERICO AT AT E OMERI O TA E I O FATE A AMERIC I S IN A BOY SOME DAY HE COULD BE PRESENT UNITED STATES HERE WE ARE TO DAT KOPHE THERE IS THIS HUGE AMOUNT OF HOPE THAT HE BRINGS TO THIS COUNTRY TO THE POLITICAL 

In [7]:
print(collection_of_text)
final_complete_speech = ""

# convert batch of text into one complete sentence
for i in collection_of_text:
    final_complete_speech += i

print(final_complete_speech)

["THE DAY OF THE SPEECH I COULDN'T EAT ALL DAY I WAS SO NERVOUS FOR HIM I WEN IT IT TO GO JUST PERFECTLY HE STARTS OFF AND I GET O TELL YOU IAM SWEAT AND I FEEL LIKE I'M IN A SANA I STAND HERE KNOWING THAT MY STORY IS PART OF THE LARGER AMERICAN STORY THAT I OWE ADEBT TO ALL OF THOSE WHO CAME BEFORE ME AND THAT NO OTHER COUNTRY ON EARTH IS MY STORY EVEN POSIBLIPEEPLE GET THIN TO ITAND SLOWLY BUT SURELY GOES FROM GOOD TO GREAT IT IS TAT UNDOMENTAL BELIEF I AM MY ROTHER'S KEEPER I AM MY THI TERS KEEPER THAT MAKES THIS OT REWORK AND THEN I START TO GET THIS FEELING OF RELIEF THAT HE'S DOING A GOOD JOB AND THEN WHEN HE JUST HIT A HOME RUN WITH IT TO NIGHT THERE IS NOT A LIBERAL AMERICA AND A CONSERVATIVE AMERICA THERE IS THE UNITED STATES OF AMERICA A A E MERI A AN A MY O MERIO A OINAL MERICO AT AT E OMERI O TA E I O FATE A AMERIC I S IN A BOY SOME DAY HE COULD BE PRESENT UNITED STATES HERE WE ARE TO DAT KOPHE THERE IS THIS HUGE AMOUNT OF HOPE THAT HE BRINGS TO THIS COUNTRY TO THE POLITICA